In [1]:
import sys
sys.path.insert(0, "/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/inference.ipynb")

from src.retrievers.bm25colbert import BM25ColBertRetriever
from src.retrievers.bm25e5 import BM25E5Retriever
from src.readers.fid import FiDReader
from src.retrievers.e5 import E5Retriever

import torch
from tqdm import tqdm

In [10]:
TUNED_READER_PATH = '/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/logs/join_e5_fid_triviaqa/reader_bestmodel.pt'
BASE_PATH = '/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/data/bases/e5_scipdf_base'
READER_INPUT_FORMAT = "context: {c}\n\nquestion: {q}"
READER_GEN_ML = 1024

QUESTIONS = [
    "RETRO method explanation",
    "kNN-LM method explanation",
    "DPR method explanation",
    "RAG method explanation",
    "FiD method explanation",
    "EMDR2 method",
    "Atlas method explanation",
    "REPLUG method explanation",
    "ColBERT method explanation"
]

In [11]:
def inference(reader, retriever, queries):
    answers = []
    for query in tqdm(queries):
        print("QUERY: ", query)
        texts, k_scores, metadata = retriever.search(query)

        print("CONTEXTS:\n", '\n\n=='.join(texts))

        formated_txts = list(map(
            lambda t: READER_INPUT_FORMAT.format(q=query,c=t), texts))

        tokenized_txts = reader.tokenize(formated_txts)
        
        cands_k = len(texts)

        # Generating Answers by predicted indices
        output = reader.model.generate(
            input_ids=tokenized_txts['input_ids'].view(1, cands_k, -1),
            attention_mask=tokenized_txts['attention_mask'].view(1, cands_k, -1), 
            max_length=READER_GEN_ML, eos_token_id=reader.tokenizer.eos_token_id,
            penalty_alpha=0.6, top_k=20, do_sample=True)
        
        predicted = reader.tokenizer.batch_decode(output, skip_special_tokens=True)
        answers += predicted

        print("ANSWER: ", predicted[0])
        print()

    return answers

In [4]:
reader = FiDReader()
reader.model.load_state_dict(torch.load(TUNED_READER_PATH))
#reader.load_model(TUNED_READER_PATH)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<All keys matched successfully>

#### E5 + FID

In [5]:
TUNED_RETRIEVER_PATH = '/home/dzigen/Desktop/ITMO/ВКР/КМУ2024/logs/join_e5_fid_triviaqa/retriever_bestmodel.pt'

e5_retriever = E5Retriever(k=20)
e5_retriever.model.load_state_dict(torch.load(TUNED_RETRIEVER_PATH))
#colb_retriever.load_model(TUNED_RETRIEVER_PATH)
e5_retriever.load_base(BASE_PATH)

Loading query E5-model...
Loading document E5-model...


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Loading precomputed e5-base...


In [12]:
inference(reader, e5_retriever, QUESTIONS)

  0%|          | 0/9 [00:00<?, ?it/s]

QUERY:  RETRO method explanation
CONTEXTS:
 Shall We Pretrain Autoregressive Language Models with Retrieval? A Comprehensive Study
To answer the above question and bridge the missing gap, we perform an extensive study on RETRO, as to the best of our knowledge, RETRO is the only retrieval-augmented autoregressive LM that supports large-scale pretraining with retrieval on the massive pretraining corpus with hundreds of billion or trillion tokens.

==Improving language models by retrieving
from trillions of tokens
Retro combines a frozen Bert retriever, a diﬀerentiable encoder and a chunked cross-attention mechanism to predict tokens based on an order of magnitude more data than what is typically consumed during training.

==Shall We Pretrain Autoregressive Language Models with Retrieval? A Comprehensive Study
Additionally, our findings demonstrate that RETRO can leverage retrieved neighbors and significantly improves accuracy for knowledgeintensive tasks in zero-shot evaluations.

==Impr

 11%|█         | 1/9 [00:30<04:01, 30.18s/it]

ANSWER:  On how to design RTRO language learning model in the context of a computer environment?

QUERY:  kNN-LM method explanation
CONTEXTS:
 COPY IS ALL YOU NEED
• kNN-LM (Khandelwal et al., 2020) is a retrieval-augmented generation model, which extends a pre-trained neural language model by linearly interpolating its next token distribution with a k-nearest neighbors (kNN) model.

==GENERALIZATION THROUGH MEMORIZATION : NEAREST NEIGHBOR LANGUAGE MODELS
The kNN-LM involves augmenting such a pre-trained LM with a nearest neighbors retrieval mechanism, without any additional training (the representations learned by the LM remain unchanged).

==GENERALIZATION THROUGH MEMORIZATION : NEAREST NEIGHBOR LANGUAGE MODELS
We introduce kNN-LM, an approach that extends a pre-trained LM by linearly interpolating its next word distribution with a k-nearest neighbors (kNN) model.

==GENERALIZATION THROUGH MEMORIZATION : NEAREST NEIGHBOR LANGUAGE MODELS
kNN-LM The keys used for kNN-LM are the 1024-di

 22%|██▏       | 2/9 [00:54<03:09, 27.01s/it]

...

ANSWER:  EXCLUDING A PROGRAM

QUERY:  RAG method explanation
CONTEXTS:
 SELF-RAG : LEARNING TO RETRIEVE , GENERATE , AND CRITIQUE THROUGH SELF-REFLECTION
Retrieval-Augmented Generation (RAG), an ad hoc approach that augments LMs with retrieval of relevant knowledge, decreases such issues.

==SELF-RAG : LEARNING TO RETRIEVE , GENERATE , AND CRITIQUE THROUGH SELF-REFLECTION
SELF-RAG is a framework that enhances the quality and factuality of an LLM through retrieval and self-reflection, without sacrificing LLM’s original creativity and versatility.

==SELF-RAG : LEARNING TO RETRIEVE , GENERATE , AND CRITIQUE THROUGH SELF-REFLECTION
While their value function simply indicates an overall score of each generation, SELF-RAG trains to an arbitrary LM to learn to generate fine-grained self-reflection and customizable inference.

==SELF-RAG : LEARNING TO RETRIEVE , GENERATE , AND CRITIQUE THROUGH SELF-REFLECTION
While we also train our model on diverse instruction-following datasets, SELF-RAG en

 44%|████▍     | 4/9 [01:41<01:59, 23.99s/it]

ANSWER:  A

QUERY:  FiD method explanation


 44%|████▍     | 4/9 [01:46<02:12, 26.59s/it]


KeyboardInterrupt: 

#### BM25ColBERT + FID

In [ ]:
TUNED_RETRIEVER_PATH = ''

colb_retriever = BM25ColBertRetriever()
colb_retriever.load_model(TUNED_RETRIEVER_PATH)
colb_retriever.load_base(BASE_PATH)

In [ ]:
inference(reader, colb_retriever, QUESTIONS)

#### BM25E5 + FID

In [ ]:
TUNED_RETRIEVER_PATH = ''

e5_retriever = BM25ColBertRetriever()
e5_retriever.load_model(TUNED_RETRIEVER_PATH)
e5_retriever.load_base(BASE_PATH)

In [ ]:
inference(reader, e5_retriever, QUESTIONS)